In [ ]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import json
import hashlib
import time
from random import randint
from time import sleep

In [ ]:
# public_key='597ec79d6258869ef487f19398a548f3'
# private_key = '2a91e39888bc470147b4da008284dc95883bbc06'
# ts = time.time()
# md = str(ts) + private_key + public_key
# hash_md = hashlib.md5(md.encode('utf-8'))

In [ ]:
# marvel_api = 'https://gateway.marvel.com/v1/public/characters?orderBy=name&apikey=' + public_key + '&hash='+ str(hash_md)
# page_api = requests.get(marvel_api)


In [ ]:
# print(page_api)

In [ ]:
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'he-IL,he;q=0.9,en-US;q=0.8,en;q=0.7',
}

def load_soup_object(url_path):
    sleep(randint(3,5))
    page = requests.get(url_path ,headers = headers)
    print(page)
    return BeautifulSoup(page.content, "html.parser")

In [ ]:
## name, current alias, affiliation, marital status, gender,
## height, weight, hair color, origin, living status, cause of death, reality, place of birth,
## identity, citenzenship, education, creators, unusual features, occupation

In [ ]:
dataframe = pd.DataFrame(columns = ['name', 'current alias', 'affiliation', 'marital status', 'gender', 'hair color',
                                    'origin', 'living status', 'reality', 'place of birth',
                                    'identity', 'citenzenship', 'education', 'creators', 'occupation'])

name_lst = [] 
current_alias_lst = [] 
affiliation_lst = [] 
marital_status_lst = [] 
gender_lst = []
hair_color_lst = [] 
origin_lst = [] 
living_status_lst = []
reality_lst = []
place_of_birth_lst = []
identity_lst = [] 
citenzenship_lst = [] 
education_lst = []
creators_lst = [] 
occupation_lst = []

url_prefix = 'https://marvel.fandom.com'
first_page = True

In [ ]:
soup = load_soup_object('https://marvel.fandom.com/wiki/Category:Characters')
not_last_page = True
while not_last_page:
    current_page_character_list = soup.find_all('div', {'class': 'category-page__members-wrapper'})
    if first_page:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})[18:]
        first_page = False
    else:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})

    for i,X in enumerate(links):     
        print(str(i) + ". " + url_prefix + X.get('href'))
        character_page_soup = load_soup_object(url_prefix + X.get('href'))
        if character_page_soup.find('div',  {'data-source': 'Name'}) is not None:
            name_content = character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            name_lst.append(name_content)
        else:
            name_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'CurrentAlias'}) is not None:
            alias_content = character_page_soup.find('div',  {'data-source': 'CurrentAlias'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            current_alias_lst.append(alias_content)
        else:
            current_alias_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Affiliation'}) is not None:
            aff_content = character_page_soup.find('div',  {'data-source': 'Affiliation'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            affiliation_lst.append(aff_content)
        else:
            affiliation_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'MaritalStatus'}) is not None:
            marital_status_string = ""
            for x in character_page_soup.find('div', {'data-source': 'MaritalStatus'}).find_all('a'):
                marital_status_string += str(x.text) + " "

            marital_status_lst.append(marital_status_string)
        else:
            marital_status_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Reality'}) is not None: 
            reality_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Reality'}).find_all('a'):
                reality_string += str(x.text) + " "
            reality_lst.append(reality_string)
        else:
            reality_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Gender'}) is not None:
            gender_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Gender'}).find_all('a'):
                gender_string += str(x.text) + " "
            gender_lst.append(gender_string)
        else:
            gender_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Hair'}) is not None:
            hair_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Hair'}).find_all('a'):
                hair_string += str(x.text) + " "
            hair_color_lst.append(hair_string)
        else:
            hair_color_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Origin'}) is not None:
            origin_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Origin'}).find_all('a'):
                origin_string += str(x.text) + " "
            origin_lst.append(origin_string)
        else:
            origin_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Status'}) is not None:
            status_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Status'}).find_all('a'):
                status_string += str(x.text) + " "
            living_status_lst.append(status_string)
        else:
            living_status_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'PlaceOfBirth'}) is not None:
            birth_string = ""
            for x in character_page_soup.find('div', {'data-source': 'PlaceOfBirth'}).find_all('a'):
                birth_string += str(x.text) + " "
            place_of_birth_lst.append(birth_string)
        else:
            place_of_birth_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Identity'}) is not None:
            identity_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Identity'}).find_all('a'):
                identity_string += str(x.text) + " "
            identity_lst.append(identity_string)
        else:
            identity_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Citizenship'}) is not None:
            #test_soup.find('div', {'data-source': 'Citizenship'}).get_text().strip()
            citizen_content = character_page_soup.find('div',  {'data-source': 'Citizenship'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            citenzenship_lst.append(citizen_content)
        else:
            citenzenship_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Education'}) is not None:
            #test_soup.find('div', {'data-source': 'Education'}).a.get_text().strip()
            educ_content = character_page_soup.find('div', {'data-source': 'Education'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            education_lst.append(educ_content)
        else:
            education_lst.append(None)


        if character_page_soup.find('div',  {'data-source': 'Occupation'}) is not None:
            occu_content = character_page_soup.find('div',  {'data-source': 'Occupation'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            occupation_lst.append(occu_content)
        else:
            occupation_lst.append(None)

        if character_page_soup.find('div',  {'data-source': 'Creators'}) is not None:
            creator_string = ""
            for x in character_page_soup.find('div', {'data-source': 'Creators'}).find_all('a'):
                creator_string += str (x.text) + " "
            creators_lst.append(creator_string)
        else:
            creators_lst.append(None)
            
    if soup.find('a', {'class':'category-page__pagination-next'}) is not None:
        next_page = soup.find('a', {'class':'category-page__pagination-next'}).get('href')
        soup = load_soup_object(next_page)
    else:
        not_last_page = False

In [ ]:
dataframe['name'] = name_lst
dataframe['current alias'] = current_alias_lst
dataframe['affiliation'] = affiliation_lst
dataframe['marital status'] = marital_status_lst
dataframe['gender'] = gender_lst
dataframe['hair color'] = hair_color_lst
dataframe['origin'] = origin_lst
dataframe['living status'] = living_status_lst
dataframe['reality'] = reality_lst
dataframe['place of birth'] = place_of_birth_lst
dataframe['identity'] = identity_lst
dataframe['citenzenship'] = citenzenship_lst
dataframe['education'] = education_lst
dataframe['creators'] = creators_lst
dataframe['occupation'] = occupation_lst

In [ ]:
dataframe

In [ ]:
dataframe.to_csv('marvel.csv')